# Predicción de Default de Clientes con Regresión Logística

En este cuaderno desarrolla un modelo de clasificación para predecir si un cliente
incurrirá en default el próximo mes. El dataset incluye 23 variables relacionadas
con historial crediticio, pagos y características demográficas.

In [7]:
import pandas as pd
import numpy as np
import os
import gzip
import pickle
import json

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    precision_score, balanced_accuracy_score, recall_score,
    f1_score, confusion_matrix
)


# Paso 1.
# Realice la limpieza de los datasets:
# - Renombre la columna "default payment next month" a "default".
# - Remueva la columna "ID".
# - Elimine los registros con informacion no disponible.
# - Para la columna EDUCATION, valores > 4 indican niveles superiores
#   de educación, agrupe estos valores en la categoría "others".
# - Renombre la columna "default payment next month" a "default"
# - Remueva la columna "ID".
#

In [8]:
# Cargar datos
train = pd.read_csv("../files/input/train_data.csv.zip", index_col = False, compression = "zip")
test= pd.read_csv("../files/input/test_data.csv.zip", index_col = False, compression = "zip")
# Renombrar columna objetivo
train.rename(columns={"default payment next month": "default"}, inplace=True)
test.rename(columns={"default payment next month": "default"}, inplace=True)

# Eliminar columna ID
train.drop(columns=["ID"], inplace=True)
test.drop(columns=["ID"], inplace=True)

# Remover datos faltantes
train.dropna(inplace=True)
test.dropna(inplace=True)

# Agrupar niveles poco comunes de EDUCATION
train["EDUCATION"] = train["EDUCATION"].clip(upper=4)
test["EDUCATION"] = test["EDUCATION"].clip(upper=4)


# Paso 2.
# Divida los datasets en x_train, y_train, x_test, y_test.

In [9]:
x_train = train.drop(columns=["default"])
y_train = train["default"]

x_test = test.drop(columns=["default"])
y_test = test["default"]


# Paso 3.
# Cree un pipeline para el modelo de clasificación. Este pipeline debe
# contener las siguientes capas:
# - Transforma las variables categoricas usando el método
#   one-hot-encoding.
# - Descompone la matriz de entrada usando PCA. El PCA usa todas las componentes.
# - Estandariza la matriz de entrada.
# - Selecciona las K columnas mas relevantes de la matrix de entrada.
# - Ajusta una maquina de vectores de soporte (svm).
#

In [10]:
categorical = ["SEX", "EDUCATION", "MARRIAGE"]
numeric = [col for col in x_train.columns if col not in categorical]

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical),
        ("num", StandardScaler(), numeric),
    ]
)

pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("pca", PCA()),
    ("selector", SelectKBest(score_func=f_classif)),
    ("svm", SVC())
])


# Paso 4.
# Optimice los hiperparametros del pipeline usando validación cruzada.
# Use 10 splits para la validación cruzada. Use la función de precision
# balanceada para medir la precisión del modelo.
#

In [11]:
param_grid = {
    "selector__k": [10, 15, 20],
    "svm__C": [0.1, 1, 10],
    "svm__kernel": ["rbf", "poly"],
}

grid = GridSearchCV(
    pipeline,
    param_grid,
    scoring="balanced_accuracy",
    cv=10,
    n_jobs=-1,
    verbose=1
)

grid.fit(x_train, y_train)


Fitting 10 folds for each of 18 candidates, totalling 180 fits


,estimator,"Pipeline(step...svm', SVC())])"
,param_grid,"{'selector__k': [10, 15, ...], 'svm__C': [0.1, 1, ...], 'svm__kernel': ['rbf', 'poly']}"
,scoring,'balanced_accuracy'
,n_jobs,-1
,refit,True
,cv,10
,verbose,1
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('cat', ...), ('num', ...)]"


# Paso 5.
# Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
# Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.

In [12]:
os.makedirs("files/models", exist_ok=True)

with gzip.open("files/models/model.pkl.gz", "wb") as f:
    pickle.dump(grid, f)


# Paso 6.
# Calcule las metricas de precision, precision balanceada, recall,
# y f1-score para los conjuntos de entrenamiento y prueba.
# Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# Este diccionario tiene un campo para indicar si es el conjunto
# de entrenamiento o prueba. Por ejemplo:
#
# {'dataset': 'train', 'precision': 0.8, 'balanced_accuracy': 0.7, 'recall': 0.9, 'f1_score': 0.85}
# {'dataset': 'test', 'precision': 0.7, 'balanced_accuracy': 0.6, 'recall': 0.8, 'f1_score': 0.75}
#

In [13]:
def compute_metrics(model, X, y, name):
    y_pred = model.predict(X)
    return {
        "type": "metrics",
        "dataset": name,
        "precision": float(precision_score(y, y_pred)),
        "balanced_accuracy": float(balanced_accuracy_score(y, y_pred)),
        "recall": float(recall_score(y, y_pred)),
        "f1_score": float(f1_score(y, y_pred)),
    }

metrics = [
    compute_metrics(grid, x_train, y_train, "train"),
    compute_metrics(grid, x_test, y_test, "test"),
]


# Paso 7.
# Calcule las matrices de confusion para los conjuntos de entrenamiento y
# prueba. Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# de entrenamiento o prueba. Por ejemplo:
#
# {'type': 'cm_matrix', 'dataset': 'train', 'true_0': {"predicted_0": 15562, "predicte_1": 666}, 'true_1': {"predicted_0": 3333, "predicted_1": 1444}}
# {'type': 'cm_matrix', 'dataset': 'test', 'true_0': {"predicted_0": 15562, "predicte_1": 650}, 'true_1': {"predicted_0": 2490, "predicted_1": 1420}}
#


In [14]:
def cm_to_dict(cm, name):
    return {
        "type": "cm_matrix",
        "dataset": name,
        "true_0": {"predicted_0": int(cm[0,0]), "predicted_1": int(cm[0,1])},
        "true_1": {"predicted_0": int(cm[1,0]), "predicted_1": int(cm[1,1])},
    }

metrics.append(cm_to_dict(confusion_matrix(y_train, grid.predict(x_train)), "train"))
metrics.append(cm_to_dict(confusion_matrix(y_test, grid.predict(x_test)), "test"))


In [15]:
os.makedirs("files/output", exist_ok=True)

with open("files/output/metrics.json", "w") as f:
    json.dump(metrics, f, indent=4)
